In [1]:
import CaboCha

cabocha = CaboCha.Parser('-n1')


def parse_sentence(sentence_str, sentence_begin):
    tree = cabocha.parse(sentence_str)

    offset = sentence_begin
    text = sentence_str
    for i in range(tree.chunk_size()):
        chunk = tree.chunk(i)
        chunk_begin = None

        print('chunk:')

        for j in range(
                chunk.token_pos,
                chunk.token_pos + chunk.token_size):
            token = tree.token(j)
            features = token.feature.split(',')
            token_begin = text.find(token.surface) + offset
            token_end = token_begin + len(token.surface)
            if chunk_begin is None:
                chunk_begin = token_begin

            print('    token_begin:', token_begin)
            print('    token_end:',   token_end)
            print('    features:',    features)
            print('    lemma:',       features[-3])
            print('    POS:',         features[0])
            print('    POS2:',        features[1])
            print('    NE:',          token.ne)
            print()

            text = text[token_end-offset:]
            offset = token_end

        chunk_end = token_end

        print('  chunk_link:',    chunk.link)
        print('  chunk_begin:',   chunk_begin)
        print('  chunk_end:',     chunk_end)
        print()



parse_sentence('プログラムを作って、動かしながら自然言語処理を学ぶ。', 0)


chunk:
    token_begin: 0
    token_end: 5
    features: ['名詞', 'サ変接続', '*', '*', '*', '*', 'プログラム', 'プログラム', 'プログラム']
    lemma: プログラム
    POS: 名詞
    POS2: サ変接続
    NE: O

    token_begin: 5
    token_end: 6
    features: ['助詞', '格助詞', '一般', '*', '*', '*', 'を', 'ヲ', 'ヲ']
    lemma: を
    POS: 助詞
    POS2: 格助詞
    NE: O

  chunk_link: 1
  chunk_begin: 0
  chunk_end: 6

chunk:
    token_begin: 6
    token_end: 8
    features: ['動詞', '自立', '*', '*', '五段・ラ行', '連用タ接続', '作る', 'ツクッ', 'ツクッ']
    lemma: 作る
    POS: 動詞
    POS2: 自立
    NE: O

    token_begin: 8
    token_end: 9
    features: ['助詞', '接続助詞', '*', '*', '*', '*', 'て', 'テ', 'テ']
    lemma: て
    POS: 助詞
    POS2: 接続助詞
    NE: O

    token_begin: 9
    token_end: 10
    features: ['記号', '読点', '*', '*', '*', '*', '、', '、', '、']
    lemma: 、
    POS: 記号
    POS2: 読点
    NE: O

  chunk_link: 4
  chunk_begin: 6
  chunk_end: 10

chunk:
    token_begin: 10
    token_end: 13
    features: ['動詞', '自立', '*', '*', '五段・サ行', '連用形', '動かす', 'ウゴカシ

In [2]:
import cabochaparser   as parser
import sqlitedatastore as datastore


datastore.connect()
for doc_id in datastore.get_all_ids(limit=-1):
    row = datastore.get(doc_id, fl=['content'])
    text = row['content']
    sentences, chunks, tokens = parser.parse(text)
    print('parsed: doc_id =', doc_id)

    datastore.set_annotation(doc_id, 'sentence', sentences)
    datastore.set_annotation(doc_id, 'chunk',    chunks)
    datastore.set_annotation(doc_id, 'token',    tokens)

datastore.close()


parsed: doc_id = 1
parsed: doc_id = 2
parsed: doc_id = 3
parsed: doc_id = 4
parsed: doc_id = 5
parsed: doc_id = 6
parsed: doc_id = 7
parsed: doc_id = 8
parsed: doc_id = 9
parsed: doc_id = 10
parsed: doc_id = 11
parsed: doc_id = 12


In [4]:
import sqlitedatastore as datastore

datastore.connect()
for doc_id in datastore.get_all_ids(limit=15):
    row = datastore.get(doc_id, fl=['content'])
    text = row['content']

    print('tokens:')
    for token in datastore.get_annotation(doc_id, 'token'):
        print('  ', token['POS'], '\t', text[token['begin']:token['end']])

    print('chunks:')
    chunks = datastore.get_annotation(doc_id, 'chunk')
    for chunk in chunks:
        _, link = chunk['link']
        print('  ', text[chunk['begin']:chunk['end']])
        if link != -1:
            parent = chunks[link]
            print('\t-->', text[parent['begin']:parent['end']])
        else:
            print('\t-->', 'None')

    print('sentences:')
    for sent in datastore.get_annotation(doc_id, 'sentence'):
        print('  ', text[sent['begin']:sent['end']])

datastore.close()


tokens:
   名詞 	 フィジー
   名詞 	 フィジー
   名詞 	 共和
   名詞 	 国
   記号 	 （
   名詞 	 フィジー
   名詞 	 きょう
   助詞 	 わ
   動詞 	 こく
   記号 	 ）
   記号 	 、
   名詞 	 通称
   名詞 	 フィジー
   助詞 	 は
   記号 	 、
   名詞 	 オセアニア
   助詞 	 の
   名詞 	 国家
   助詞 	 で
   記号 	 、
   名詞 	 イギリス
   名詞 	 連邦
   名詞 	 加盟
   名詞 	 国
   助動詞 	 で
   助動詞 	 ある
   名詞 	 首都
   助詞 	 は
   名詞 	 ビティレブ
   名詞 	 島
   助詞 	 の
   名詞 	 スバ
   名詞 	 南太平洋
   助詞 	 の
   名詞 	 フィジー
   名詞 	 諸島
   助詞 	 と
   名詞 	 北
   助詞 	 に
   名詞 	 500
   名詞 	 km
   助詞 	 程
   動詞 	 離れ
   助動詞 	 た
   名詞 	 保護
   名詞 	 領
   助詞 	 の
   名詞 	 ロツマ
   名詞 	 島
   助詞 	 に
   名詞 	 位置
   動詞 	 する
   名詞 	 群島
   名詞 	 国家
   助動詞 	 で
   助動詞 	 ある
   名詞 	 300
   名詞 	 余
   助詞 	 の
   名詞 	 火山
   名詞 	 島
   助詞 	 と
   名詞 	 珊瑚礁
   助詞 	 から
   動詞 	 なる
   名詞 	 西
   助詞 	 に
   名詞 	 バヌアツ
   記号 	 、
   名詞 	 東
   助詞 	 に
   名詞 	 トンガ
   記号 	 、
   名詞 	 北
   助詞 	 に
   名詞 	 ツバル
   助詞 	 が
   動詞 	 ある
   名詞 	 目次
   名詞 	 国名
   名詞 	 [
   名詞 	 編集
   名詞 	 ]
   名詞 	 正式
   名詞 	 名称
   助詞 	 は
   記号 	 、
   名詞 	 Republic
   名詞 	 of
   名詞 	 Fiji
   